In [5]:
import pandas as pd # needed for most operation
import numpy as np # needed for some array operations
#from sqlalchemy import create_engine # needed for DB connection

inputfile = 'BI_Raw_Data.csv'

# Read the CSV file into a pandas dataframe
df = pd.read_csv(inputfile, delimiter=';', encoding="ISO-8859-1")

In [9]:
# Create the product table
product = df[['Product_Name', 'Product_Category']].drop_duplicates()
product['productid'] = np.arange(1, len(product)+1)
product = product.rename(columns={'Product_Name': 'productname', 'Product_Category': 'category'})
product = product[['productid', 'productname', 'category']]

In [11]:
# Create the customer table
customer = df[['Customer_Name', 'Customer_Country']].drop_duplicates()
customer['customerid'] = np.arange(1, len(customer)+1)
customer = customer.rename(columns={'Customer_Name': 'customername', 'Customer_Country': 'country'})
customer = customer[['customerid', 'customername', 'country']]

In [19]:

# Create the sales table
sales = df[['Order_Date_Year', 'Order_Date_Month', 'Order_Date_Day', 'Customer_Name', 'Product_Name', 'Product_Order_Price_Total']]

# Ensure columns are called the same before merging
# sales = sales.rename(columns={'Customer_Name': 'customername'})
# sales = sales.rename(columns={'Product_Name': 'productname'})

sales = sales.merge(customer, on='customername', how='inner')
sales = sales.merge(product, on='productname', how='inner')
sales = sales.rename(columns={'Order_Date_Year': 'year', 'Order_Date_Month': 'month', 'Order_Date_Day': 'day', 'Product_Order_Price_Total': 'sales'})
sales = sales[['year', 'month', 'day', 'customerid', 'productid', 'sales']]

KeyError: 'category'

In [15]:
# Output the resulting tables to CSV files
customer.to_csv('output/customer.csv', index=False)
product.to_csv('output/product.csv', index=False)
sales.to_csv('output/sales.csv', index=False)